# Import libs

In [ ]:
!nvidia-smi

Sun Aug 15 12:25:26 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
pip install --upgrade transformers

     |████████████████████████████████| 2.6 MB 7.9 MB/s 
     |████████████████████████████████| 3.3 MB 49.4 MB/s 
     |████████████████████████████████| 636 kB 52.6 MB/s 
     |████████████████████████████████| 895 kB 47.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install simpletransformers

     |████████████████████████████████| 221 kB 7.3 MB/s 
     |████████████████████████████████| 1.2 MB 57.0 MB/s 
     |████████████████████████████████| 8.0 MB 27.6 MB/s 
     |████████████████████████████████| 1.6 MB 62.5 MB/s 
     |████████████████████████████████| 264 kB 55.4 MB/s 
     |████████████████████████████████| 124 kB 70.8 MB/s 
     |████████████████████████████████| 43 kB 2.8 MB/s 
     |████████████████████████████████| 170 kB 60.3 MB/s 
     |████████████████████████████████| 97 kB 9.3 MB/s 
     |████████████████████████████████| 133 kB 69.0 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
     |████████████████████████████████| 243 kB 60.4 MB/s 
     |████████████████████████████████| 118 kB 75.5 MB/s 
     |████████████████████████████████| 75 kB 4.8 MB/s 
     |████████████████████████████████| 111 kB 63.5 MB/s 
     |████████████████████████████████| 4.2 MB 54.2 MB/s 
     |████████████████████████████████| 122 kB 75.1 MB/s 
     |█████████████████

In [ ]:
# !pip install fastbpe
# !pip install fairseq

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import torch
import pandas as pd
import logging
import wandb
import os
from sklearn.model_selection import train_test_split

cuda_available = torch.cuda.is_available()

# Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir('/content/gdrive/My Drive/BERT/SA')

Mounted at /content/gdrive


In [ ]:
# !wget https://public.vinai.io/PhoBERT_base_fairseq.tar.gz
# !tar -xzvf PhoBERT_base_fairseq.tar.gz

In [ ]:
# from fairseq.models.roberta import RobertaModel
# from fairseq.data.encoders.fastbpe import fastBPE  
# from fairseq import options  
# import numpy as np
# import random

# # Khởi tạo Byte Pair Encoding cho PhoBERT
# class BPE():
#   bpe_codes = 'PhoBERT_base_fairseq/bpe.codes'
# args = BPE()

# phoBERT = RobertaModel.from_pretrained('PhoBERT_base_fairseq', checkpoint_file='model.pt')
# phoBERT.bpe = fastBPE(args) #Incorporate the BPE encoder into PhoBERT

In [ ]:
# def generate_aug_text(seed):
#     gen_sentence = []
#     words = seed.split()
#     random.seed(2)

#     seed = " ".join(words)

#     for i in range(len(words)):

#         if random.randint(0, 1):
#             continue
        
#         temp = words.copy()
#         save = temp[i]
#         temp[i] = ' <mask>'

#         text_masked_tok = ' '.join(temp)
#         # print("sen: ", text_masked_tok)
        
#         topk_filled_outputs = phoBERT.fill_mask(text_masked_tok, topk=1) 
#         # topk_probs = [item[1] for item in topk_filled_outputs]
#         # print('Total probability: ', np.sum(topk_probs))

#         # print('Top 10 in mask: ')
#         # for i, output in enumerate(topk_filled_outputs): 
#         output = topk_filled_outputs
#         content = output[0][0]
#         prob = output[0][1]
#         word = output[0][2]
        
#         if word == save or (prob < 0.15 or prob > 0.5) or len(word) < 2:
#             continue
#             # print("pingping: ", word)

#         gen_sentence.append([content, prob])


#     gen_sentence = sorted(gen_sentence ,key=lambda l:l[1], reverse = True)

#     if len(gen_sentence) == 0: 
#         return 0

#     elif len(gen_sentence) > 2:
#         gen_sentence = gen_sentence[:2]

#     gen_sentence = [i[0] for i in gen_sentence]
#     print("-")
#     return gen_sentence

# Read data 

In [ ]:
df = pd.read_csv('preprocessed.csv', index_col= 0)
df = df.drop('Unnamed: 0.1', axis = 1)
df.head()

,content,label,avg_length,sen_word_seg
0,Nghệ sỹ hài Dương Hoài Linh bị chỉ trích vì gi...,1,1683,Nghệ_sỹ hài Dương_Hoài_Linh bị chỉ_trích vì gi...
1,Người dân bị lập biên bản vì không muốn làm tr...,1,1588,Người_dân bị lập biên_bản vì không muốn làm tr...
2,Ngân hàng thế giới nói dòng vốn đầu tư trực ti...,1,1412,Ngân_hàng thế_giới nói dòng vốn đầu_tư trực_ti...
3,Người Việt uống nhiều bia rượu hơn trong đại d...,1,1017,Người Việt uống nhiều bia rượu hơn trong đại_d...
4,Nhiều người Việt Nam muốn đi Hoa Kỳ tiêm vacci...,1,1402,Nhiều người Việt_Nam muốn đi Hoa_Kỳ tiêm vacci...


In [ ]:
df = df[['sen_word_seg', 'label']]
df['sen_word_seg'] = df['sen_word_seg'].astype('category')
df = df.rename(columns = {'sen_word_seg': 'text', 'label': 'labels'})
df

,text,labels
0,Nghệ_sỹ hài Dương_Hoài_Linh bị chỉ_trích vì gi...,1
1,Người_dân bị lập biên_bản vì không muốn làm tr...,1
2,Ngân_hàng thế_giới nói dòng vốn đầu_tư trực_ti...,1
3,Người Việt uống nhiều bia rượu hơn trong đại_d...,1
4,Nhiều người Việt_Nam muốn đi Hoa_Kỳ tiêm vacci...,1
...,...,...
10179,Thủ_tướng Nguyễn_Xuân_Phúc dự Hội_nghị tổng_kế...,0
10180,Chủ_tịch Quốc_hội Nguyễn_Thị_Kim_Ngân chủ_trì ...,0
10181,Chủ_tịch nước Trần_Đại_Quang dự chương_trình n...,0
10182,Ông Hầu_A_Lềnh giữ chức Phó chủ_tịch - Tổng_th...,0


# Split data

In [ ]:
df['labels'].value_counts()

0    30451
1    19066
Name: labels, dtype: int64

In [ ]:
df_1 = df[df['labels'] == 1]
df_0 = df[df['labels'] == 0]

# df_1 = df_1.sample(frac=1).reset_index(drop=True)
# df_0 = df_0.sample(frac=1).reset_index(drop=True)

test_df = pd.concat([df_0.iloc[-5000:, :], df_1.iloc[-5000:, :]])
test_df = test_df.sample(frac=1, random_state = 5).reset_index(drop=True)

train_subset_df = pd.concat([df_0.iloc[:-9000, :], df_1.iloc[:-5000, :]])
train_subset_df = train_subset_df.sample(frac=1, random_state = 5).reset_index(drop=True)

In [ ]:
print(train_subset_df['labels'].value_counts())
print(test_df['labels'].value_counts())
print(train_subset_df.shape)
#0.6, 0.4

0    21451
1    14066
Name: labels, dtype: int64
1    5000
0    5000
Name: labels, dtype: int64
(35517, 2)


In [ ]:
w_1 = 35517 / (2 * 12066)
w_0 = 35517 / (2 * 21451)
w_0

0.8278635028669992

# Cross validation

## Define model

In [ ]:
# model = ClassificationModel( "xlmroberta", "microsoft/Multilingual-MiniLM-L12-H384", 
#                                 args= create_args_model_2(), 
#                                 use_cuda = cuda_available, 
#                                 num_labels = 2,  
#                                 weight = [1.5, 0.8])

# for para in model.get_named_parameters():
#     print(para)

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# MODEL 1 ARGS: tuning all classifier --------------------------
model_args = {
    'num_train_epochs': 2,
    # 'save_eval_checkpoints': False,
    'train_batch_size': 40,
    # 'eval_batch_size': 256,
    'overwrite_output_dir': True,
    # 'sliding_window': True,
    'use_early_stopping': True,
    'no_cache': True,
    'max_seq_length': 300,
    'wandb_project': 'nlp_bert cross validation (early stopping)',
    'early_stopping_consider_epochs': True,
    'save_steps': -1,  # don't save model every number of steps


    # only cross validation, try not to save on disk-----------------
    'save_model_every_epoch': False,
    'save_optimizer_and_scheduler': False,
    'save_eval_checkpoints': False,
    'no_save': True,

    # test early stopping
    'evaluate_during_training': True,
    'early_stopping_delta': 0.01,
    'evaluate_during_training_steps': 1000,
    'evaluate_during_training_verbose': True,
    'early_stopping_patience': 5,
}

# MODEL 2 ARGS: only tuning in classifier layer---------------------------------------
    # increase adam epsilon to slower the optimizer
    # 'adam_epsilon' : 1e-6

def create_args_model_2():
    model_args = ClassificationArgs()

    # Set custom weight bias to train only
    model_args.custom_parameter_groups = [
        {
            "params": ["classifier.dense.weight", 
                       "classifier.dense.bias"],
            "lr": 1e-3,
        }
    ]
    model_args.train_custom_parameters_only = True
    

    model_args.evaluate_during_training = True 
    model_args.num_train_epochs = 3
    model_args.train_batch_size: 40
    model_args.overwrite_output_dir = True
    model_args.use_early_stopping = True
    model_args.no_cache = True
    model_args.max_seq_length = 300
    model_args.wandb_project = 'nlp_bert cross validation (only last layer tuning)'
    model_args.early_stopping_consider_epochs = True
    model_args.save_steps = -1  # don't save model every number of steps


    # only cross validation, try not to save on disk-----------------
    model_args.save_model_every_epoch = False
    model_args.save_optimizer_and_scheduler = False
    model_args.save_eval_checkpoints = False
    model_args.no_save = True

    # test early stopping
    # model_args.early_stopping_delta = 0.01    # %decrease in loss function
    # model_args.early_stopping_patience = 5  # % wait 5 evaluation < delta to stop 
    model_args.evaluate_during_training_steps = 2000   #takes 1000 steps then evaluate
    
    # model_args.early_stopping_metric = "f1"
    # model_args.early_stopping_metric_minimize = False

    return model_args

# MODEL v1 ARGS: only tuning in layers 0, 10, 11, classifier
# default lr + weight decay = 0---------------------------------------
def create_args_model_2():
    model_args = ClassificationArgs()

    # Set custom weight bias to train only
    model_args.custom_parameter_groups = [
        {
            "params": ["classifier.dense.weight", 
                       "classifier.dense.bias"],
            "lr": 1e-5
        }
    ]

    model_args.custom_layer_parameters = [
        {
            "layer": 10,
            "lr": 1e-5,
        },

        {
            "layer": 11,
            "lr": 1e-5,
        },

        {
            "layer": 0,
            "lr": 1e-5,
        },
    ]

    model_args.train_custom_parameters_only = True
    

    model_args.evaluate_during_training = True 
    model_args.num_train_epochs = 3
    model_args.train_batch_size: 40
    model_args.overwrite_output_dir = True
    model_args.use_early_stopping = True
    model_args.no_cache = True
    model_args.max_seq_length = 300
    model_args.wandb_project = 'nlp_bert cross validation (layer 0 - 10 - 11 - classifier layers tuning)'
    model_args.early_stopping_consider_epochs = True
    model_args.save_steps = -1  # don't save model every number of steps


    # only cross validation, try not to save on disk-----------------
    model_args.save_model_every_epoch = False
    model_args.save_optimizer_and_scheduler = False
    model_args.save_eval_checkpoints = False
    model_args.no_save = True

    # test early stopping
    # model_args.early_stopping_delta = 0.01    # %decrease in loss function
    # model_args.early_stopping_patience = 5  # % wait 5 evaluation < delta to stop 
    model_args.evaluate_during_training_steps = 1000   #takes 1000 steps then evaluate
    
    # model_args.early_stopping_metric = "f1"
    # model_args.early_stopping_metric_minimize = False

    return model_args


# MODEL v2: ARGS  : only tuning in layers 0, 10, 11, classifier: 
# default lr + weight decay = 0.01 ---------------------------------------
def create_args_model_2():
    model_args = ClassificationArgs()

    # Set custom weight bias to train only
    model_args.custom_parameter_groups = [
        {
            "params": ["classifier.dense.weight", 
                       "classifier.dense.bias"],
            "lr": 1e-5
        }
    ]

    model_args.custom_layer_parameters = [
        {
            "layer": 10,
            "lr": 1e-5,
        },

        {
            "layer": 11,
            "lr": 1e-5,
        },

        {
            "layer": 0,
            "lr": 1e-5,
        },
    ]

    model_args.train_custom_parameters_only = True
    

    model_args.evaluate_during_training = True 
    model_args.num_train_epochs = 3
    model_args.train_batch_size: 40
    model_args.overwrite_output_dir = True
    model_args.use_early_stopping = True
    model_args.no_cache = True
    model_args.max_seq_length = 300
    model_args.wandb_project = 'nlp_bert cross validation (layer 0 - 10 - 11 - classifier layers tuning)'
    model_args.early_stopping_consider_epochs = True
    model_args.save_steps = -1  # don't save model every number of steps


    # only cross validation, try not to save on disk-----------------
    model_args.save_model_every_epoch = False
    model_args.save_optimizer_and_scheduler = False
    model_args.save_eval_checkpoints = False
    model_args.no_save = True

    # test early stopping
    # model_args.early_stopping_delta = 2    # %decrease in loss function
    # model_args.early_stopping_patience = 5  # % wait 5 evaluation < delta to stop 
    model_args.evaluate_during_training_steps = 1000   #takes 1000 steps then evaluate
    
    model_args.weight_decay = 0.01
    # model_args.early_stopping_metric = "f1"
    # model_args.early_stopping_metric_minimize = False

    return model_args

# MODEL v3, 4, 6 ARGS: only tuning in layers 0, 10, 11, classifier---------------------------------------
def create_args_model_2():
    model_args = ClassificationArgs()

    model_args.wandb_project = 'nlp_bert cross validation (layer 0 - 10 - 11 - classifier layers tuning)'
    model_args.num_train_epochs = 4

    # Set custom weight bias to train only
    model_args.custom_parameter_groups = [
        {
            "params": ["classifier.dense.weight"],
            "lr": 0.005868
        }, 
        {
            "params": ["classifier.dense.bias"],
            "lr": 0.004157
        }
    ]

    model_args.custom_layer_parameters = [
        {
            "layer": 10,
            "lr": 0.008397,
        },

        {
            "layer": 11,
            "lr": 0.007113,
        },

        {
            "layer": 0,
            "lr": 0.001233,
        },
    ]
    model_args.weight_decay = 0.01

    model_args.train_custom_parameters_only = True

    model_args.evaluate_during_training = True 
    model_args.train_batch_size: 40
    model_args.overwrite_output_dir = True
    model_args.use_early_stopping = True
    model_args.no_cache = True
    model_args.max_seq_length = 300
    model_args.early_stopping_consider_epochs = True
    model_args.save_steps = -1  # don't save model every number of steps


    # only cross validation, try not to save on disk-----------------
    model_args.save_model_every_epoch = False
    model_args.save_optimizer_and_scheduler = False
    model_args.save_eval_checkpoints = False
    # model_args.no_save = True

    # test early stopping
    # model_args.early_stopping_delta = 5    # %decrease in loss function
    # model_args.early_stopping_patience = 2  # % wait 5 evaluation < delta to stop 
    model_args.evaluate_during_training_steps = 2000   #takes 1000 steps then evaluate
    
    # model_args.early_stopping_metric = "f1"
    # model_args.early_stopping_metric_minimize = False

    return model_args


# MODEL v3, 4, 6 ARGS: only tuning in layers 0, 10, 11, classifier---------------------------------------
def create_args_model_2():
    model_args = ClassificationArgs()

    model_args.wandb_project = 'nlp_bert cross validation (layer 0 - 10 - 11 - classifier layers tuning)'
    model_args.num_train_epochs = 3

    # Set custom weight bias to train only
    model_args.custom_parameter_groups = [
        {
            "params": ["classifier.dense.weight"],
            "lr": 1e-5
        }, 
        {
            "params": ["classifier.dense.bias"],
            "lr": 1e-5
        }
    ]

    model_args.custom_layer_parameters = [
        {
            "layer": 10,
            "lr": 1e-5,
        },

        {
            "layer": 11,
            "lr": 1e-5,
        },

        {
            "layer": 0,
            "lr": 1e-5,
        },
    ]
    model_args.weight_decay = 0.1

    model_args.train_custom_parameters_only = True

    model_args.evaluate_during_training = True 
    model_args.train_batch_size: 40
    model_args.overwrite_output_dir = True
    model_args.use_early_stopping = True
    model_args.no_cache = True
    model_args.max_seq_length = 300
    model_args.early_stopping_consider_epochs = True
    model_args.save_steps = -1  # don't save model every number of steps


    # only cross validation, try not to save on disk-----------------
    model_args.save_model_every_epoch = False
    model_args.save_optimizer_and_scheduler = False
    model_args.save_eval_checkpoints = False
    # model_args.no_save = True

    # test early stopping
    # model_args.early_stopping_delta = 5    # %decrease in loss function
    # model_args.early_stopping_patience = 2  # % wait 5 evaluation < delta to stop 
    model_args.evaluate_during_training_steps = 2000   #takes 1000 steps then evaluate
    
    # model_args.early_stopping_metric = "f1"
    # model_args.early_stopping_metric_minimize = False

    return model_args
# 

# final model v3 - no class weight (from hyper sweep 7)---------------------------------------
def create_args_model_3():
    model_args = ClassificationArgs()

    model_args.wandb_project = 'nlp_bert cross validation (layer 0 - 10 - 11 - classifier layers tuning)'
    model_args.num_train_epochs = 3

    # Set custom weight bias to train only
    model_args.custom_parameter_groups = [
        {
            "params": ["classifier.dense.weight"],
            "lr": 0.00005054
        }, 
        {
            "params": ["classifier.dense.bias"],
            "lr": 0.00006385
        }
    ]

    model_args.custom_layer_parameters = [
        {
            "layer": 10,
            "lr": 0.00002957,
        },

        {
            "layer": 11,
            "lr": 0.00007144,
        },

        {
            "layer": 0,
            "lr": 0.00000133,
        },
    ]
    model_args.weight_decay = 0.1

    model_args.train_custom_parameters_only = True

    model_args.evaluate_during_training = True 
    model_args.train_batch_size: 40
    model_args.overwrite_output_dir = True
    model_args.use_early_stopping = True
    model_args.no_cache = True
    model_args.max_seq_length = 300
    # model_args.early_stopping_consider_epochs = True
    model_args.save_steps = -1  # don't save model every number of steps


    # only cross validation, try not to save on disk-----------------
    model_args.save_model_every_epoch = False
    model_args.save_optimizer_and_scheduler = False
    model_args.save_eval_checkpoints = False
    model_args.no_save = True

    # test early stopping
    # model_args.early_stopping_delta = 5    # %decrease in loss function
    # model_args.early_stopping_patience = 2  # % wait 5 evaluation < delta to stop 
    model_args.evaluate_during_training_steps = 2000   #takes 1000 steps then evaluate
    
    # model_args.early_stopping_metric = "f1"
    # model_args.early_stopping_metric_minimize = False

    return model_args


# final model v4 - no class weight (from hyper sweep 5)---------------------------------------
def create_args_model_4():
    model_args = ClassificationArgs()

    model_args.wandb_project = 'nlp_bert cross validation (layer 0 - 10 - 11 - classifier layers tuning)'
    model_args.num_train_epochs = 3

    # Set custom weight bias to train only
    model_args.custom_parameter_groups = [
        {
            "params": ["classifier.dense.weight"],
            "lr": 0.00003383
        }, 
        {
            "params": ["classifier.dense.bias"],
            "lr": 0.00007107
        }
    ]

    model_args.custom_layer_parameters = [
        {
            "layer": 10,
            "lr": 0.00001716,
        },

        {
            "layer": 11,
            "lr": 0.00005632,
        },

        {
            "layer": 0,
            "lr": 0.00005807,
        },
    ]
    model_args.weight_decay = 0.1

    model_args.train_custom_parameters_only = True

    model_args.evaluate_during_training = True 
    model_args.train_batch_size: 40
    model_args.overwrite_output_dir = True
    model_args.use_early_stopping = True
    model_args.no_cache = True
    model_args.max_seq_length = 300
    # model_args.early_stopping_consider_epochs = True
    model_args.save_steps = -1  # don't save model every number of steps


    # only cross validation, try not to save on disk-----------------
    model_args.save_model_every_epoch = False
    model_args.save_optimizer_and_scheduler = False
    model_args.save_eval_checkpoints = False
    model_args.no_save = True

    # test early stopping
    # model_args.early_stopping_delta = 5    # %decrease in loss function
    # model_args.early_stopping_patience = 2  # % wait 5 evaluation < delta to stop 
    model_args.evaluate_during_training_steps = 2000   #takes 1000 steps then evaluate
    
    # model_args.early_stopping_metric = "f1"
    # model_args.early_stopping_metric_minimize = False

    return model_args


# final model v5 - no class weight (from hyper sweep 1)---------------------------------------
def create_args_model_5():
    model_args = ClassificationArgs()

    model_args.wandb_project = 'nlp_bert cross validation (layer 0 - 10 - 11 - classifier layers tuning)'
    model_args.num_train_epochs = 3

    # Set custom weight bias to train only
    model_args.custom_parameter_groups = [
        {
            "params": ["classifier.dense.weight"],
            "lr": 0.00003307
        }, 
        {
            "params": ["classifier.dense.bias"],
            "lr": 0.00007101
        }
    ]

    model_args.custom_layer_parameters = [
        {
            "layer": 10,
            "lr": 0.00002278
,
        },

        {
            "layer": 11,
            "lr": 0.00007011,
        },

        {
            "layer": 0,
            "lr": 0.0000723,
        },
    ]
    model_args.weight_decay = 0.1

    model_args.train_custom_parameters_only = True

    model_args.evaluate_during_training = True 
    model_args.train_batch_size: 40
    model_args.overwrite_output_dir = True
    model_args.use_early_stopping = True
    model_args.no_cache = True
    model_args.max_seq_length = 300
    # model_args.early_stopping_consider_epochs = True
    model_args.save_steps = -1  # don't save model every number of steps


    # only cross validation, try not to save on disk-----------------
    model_args.save_model_every_epoch = False
    model_args.save_optimizer_and_scheduler = False
    model_args.save_eval_checkpoints = False
    model_args.no_save = True

    # test early stopping
    # model_args.early_stopping_delta = 5    # %decrease in loss function
    # model_args.early_stopping_patience = 2  # % wait 5 evaluation < delta to stop 
    model_args.evaluate_during_training_steps = 2000   #takes 1000 steps then evaluate
    
    # model_args.early_stopping_metric = "f1"
    # model_args.early_stopping_metric_minimize = False

    return model_args

In [ ]:
def train_cv(train_subset_df, train_index, text_index):
    # Dividing data into folds
    train_temp_df = train_subset_df.iloc[train_index]
    eval_temp_df = train_subset_df.iloc[test_index]

    # Create a ClassificationModel
    cprint((f'\n======= Initialize model ========\n'), 'green')

    # Model 1===========
    # model = ClassificationModel( "xlmroberta", "microsoft/Multilingual-MiniLM-L12-H384", 
    #                             args=model_args, 
    #                             use_cuda = cuda_available, 
    #                             num_labels = 2,  
    #                             weight = [0.8, 1.5])
    # =================


    # Model 2==========
    # model = ClassificationModel( "xlmroberta", "microsoft/Multilingual-MiniLM-L12-H384", 
    #                             args= create_args_model_2(), 
    #                             use_cuda = cuda_available, 
    #                             num_labels = 2,  
    #                             weight = [1.5, 0.8])
    
        # Model 2==========
    model = ClassificationModel( "xlmroberta", "microsoft/Multilingual-MiniLM-L12-H384", 
                                args= create_args_model_5(), 
                                use_cuda = cuda_available, 
                                num_labels = 2)
    
    #==================


    # Train model
    cprint(f'\n======= Train model ========\n', 'green')
    model.train_model(train_temp_df, eval_df = eval_temp_df, f1 = sklearn.metrics.f1_score)

    # Evaluate the model
    cprint(f'\n======= Evaluate model ========\n', 'green')
    result, model_outputs, wrong_predictions = model.eval_model(eval_temp_df, 
                            f1 = sklearn.metrics.f1_score, 
                            acc = sklearn.metrics.accuracy_score, 
                            conf = sklearn.metrics.confusion_matrix, 
                            report = sklearn.metrics.classification_report)
    
    print(result['conf'])
    print(result['report'])

    cv_results.append(result)
    wandb.log(result)

    # MODEL 2-----
    # wandb.join()
    # wandb.finish()
    # -----------

    # Free gpu memory
    del model
    del train_temp_df
    del eval_temp_df
    del model_outputs 
    del wrong_predictions

    torch.cuda.empty_cache()
    gc.collect()

    return result

In [ ]:
from sklearn.model_selection import StratifiedKFold, train_test_split
import sklearn
from tqdm.notebook import tqdm
from termcolor import colored, cprint
import wandb
import gc

k_folds = 3
kfold = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state = 5)  # fix random shuffle



cv_results = []

for fold, (train_index, test_index) in enumerate(kfold.split(train_subset_df['text'], train_subset_df['labels'])):
    cprint((f'\n================ Fold: {fold} =================\n\n'), 'white', 'on_green')

    if fold == 2:
        res = train_cv(train_subset_df, train_index, test_index)
        cv_results.append(res)

    


================ Fold: 0 =================



================ Fold: 1 =================



================ Fold: 2 =================



======= Initialize model ========



INFO:filelock:Lock 139637444323408 acquired on /root/.cache/huggingface/transformers/12a5ad52cb7fc5542e16e354fe6eb487f2f87edac63bf85dc238b1236dbaf24c.ccf88548169a21266c411bcf65585ba761d762a9c85fde572f529806fdd94ee2.lock


Downloading:   0%|          | 0.00/430 [00:00<?, ?B/s]

INFO:filelock:Lock 139637444323408 released on /root/.cache/huggingface/transformers/12a5ad52cb7fc5542e16e354fe6eb487f2f87edac63bf85dc238b1236dbaf24c.ccf88548169a21266c411bcf65585ba761d762a9c85fde572f529806fdd94ee2.lock
You are using a model of type bert to instantiate a model of type xlm-roberta. This is not supported for all configurations of models and can yield errors.
INFO:filelock:Lock 139637453208400 acquired on /root/.cache/huggingface/transformers/e1243df19ff0e1975c063fc4c531ba7fdad1ee538e0d94c41a766febfee0c8ab.3d27c3b243133a56a858d62deffdc59141c45422837cf3fde167b873bad92273.lock


Downloading:   0%|          | 0.00/471M [00:00<?, ?B/s]

...

INFO:simpletransformers.classification.classification_model: Initializing WandB run for evaluation.


Training loss,0.00026
lr,0.0
global_step,8880
_runtime,2059
_timestamp,1629032820
_step,183
tp,4618
tn,7087
fp,63
fn,71
mcc,0.97633


Training loss,▆▆▆▅▂▂▁▁▁▇▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁
lr,▂▅████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇█
_timestamp,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇█
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
tp,█▁▁▆▆▇█
tn,▁▇█████
fp,█▂▁▁▁▁▁
fn,▁██▃▃▂▁
mcc,▁▆▇████


INFO:simpletransformers.classification.classification_model:{'mcc': 0.9763346353379062, 'tp': 4618, 'tn': 7087, 'fp': 63, 'fn': 71, 'auroc': 0.9969460588462509, 'auprc': 0.9956508934751924, 'f1': 0.9856990394877269, 'eval_loss': 0.08630839570005018, 'acc': 0.9886814764760538, 'conf': array([[7087,   63],
       [  71, 4618]]), 'report': '              precision    recall  f1-score   support\n\n         0.0       0.99      0.99      0.99      7150\n         1.0       0.99      0.98      0.99      4689\n\n    accuracy                           0.99     11839\n   macro avg       0.99      0.99      0.99     11839\nweighted avg       0.99      0.99      0.99     11839\n'}


[[7087   63]
 [  71 4618]]
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99      7150
         1.0       0.99      0.98      0.99      4689

    accuracy                           0.99     11839
   macro avg       0.99      0.99      0.99     11839
weighted avg       0.99      0.99      0.99     11839

